<a href="https://colab.research.google.com/github/mrdso2-iscte/IAA_PROJECT/blob/main/IAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
data_frame=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dias_acidentes.csv', sep = ';', encoding="ISO-8859-1")
print(data_frame)

           Ano:2010           Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0               NaN                  NaN        NaN        NaN        NaN   
1      Id. Acidente             Datahora        Dia        Mês       Hora   
2        2010442610  2010:01:01 00:05:00   01/01/00   01/01/00   01/01/00   
3        2010442441  2010:01:01 00:30:00   01/01/00   01/01/00   01/01/00   
4        2010422349  2010:01:01 00:35:00   01/01/00   01/01/00   01/01/00   
...             ...                  ...        ...        ...        ...   
16378    2010470639  2010:06:29 17:30:00   01/29/00   06/01/00   01/01/00   
16379    2010468375  2010:06:29 18:00:00   01/29/00   06/01/00   01/01/00   
16380    2010470619  2010:06:29 18:00:00   01/29/00   06/01/00   01/01/00   
16381    2010462209  2010:06:29 18:00:00   01/29/00   06/01/00   01/01/00   
16382    2011502034  2010:06:29 18:15:00   01/29/00   06/01/00   01/01/00   

                         Unnamed: 5        Unnamed: 6        Unnamed: 7  \
